In [1]:
import re
from elasticsearch import Elasticsearch
import spacy
from datetime import datetime
import xml.etree.ElementTree as ET
from opencage.geocoder import OpenCageGeocode

In [3]:
# Load the spaCy models for English
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1500000 
es = Elasticsearch([{"host": "localhost", "port": 9200, "scheme": "http"}], basic_auth=('elastic', 'CbIxwM6z85Dm6fKtAJte'))
index_name = "testtt"
api_key = '8be579e383db4dc2a7f4895ebf6923d0' 
print(es.ping())

True


In [ ]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "autocomplete": {
                    "tokenizer": "autocomplete",
                    "filter": ["lowercase"]
                },
                "custom_content_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "stop",
                        "custom_stemmer"
                    ]
                }
            },
            "tokenizer": {
                "autocomplete": {
                    "type": "edge_ngram",
                    "min_gram": 3,
                    "max_gram": 20,
                    "token_chars": ["letter", "digit"]
                }
            },
            "filter": {
                "custom_stemmer": {
                    "type": "stemmer",
                    "name": "english"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "autocomplete",
                "fields": {
                    "raw": {
                        "type": "keyword"
                    }
                }
            },
            "content": {
                "type": "text",
                "analyzer": "custom_content_analyzer"
            },
            "authors": {
                "type": "nested",
                "properties": {
                    "first_name": {"type": "text"},
                    "last_name": {"type": "text"}
                }
            },
            "date": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss"
            },
            "georeferences": {
                "type": "nested",
                "properties": {
                    "expression": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword"
                            }
                        }
                    }
                }
            },
            "geopoint": {"type": "geo_point"},
            "temporal_expressions": {
                "type": "nested",
                "properties": {
                    "expression": {
                        "type": "text"
                    },
                    "date": {
                        "type": "date",
                        "format": "yyyy-MM-dd HH:mm:ss||date_optional_time"
                    }
                }
            }
        }
    }
}

es.indices.create(index=index_name, ignore=400, body=index_settings)
print(f"Index '{index_name}' created successfully.")



In [ ]:
def extract_temporal_expressions(text):
    """
    Extracts temporal expressions from the given text using spaCy.

    Args:
        text (str): Input text.

    Returns:
        list: A list of dictionaries, where each dictionary contains an "expression" key representing a temporal expression.
    """
    # Process the text using spaCy
    doc = nlp(text)
    
    # Extract entities labeled as "DATE"
    temporal_expressions = [{"expression": ent.text} for ent in doc.ents if ent.label_ == "DATE"]

    return temporal_expressions 

In [ ]:
GM &lt;GM> CANADA AND WORKERS FAR APART IN TALKS
GM CANADA, WORKERS FAR APART IN TALKS - UNION
WESTCOAST &lt;WTC> TO ACQUIRE AGIP PROPERTIES
DOLLAR STEADY IN EUROPE AHEAD OF WASHINGTON TALKS
MULFORD SAYS G-6 WANTS STABILITY
BRITISH BANKS RESIST SIGNING MEXICO PACKAGE
LATIN OIL PRODUCERS TO MEET IN CARACAS
ENCOR PLANS 125 MLN DLR DEBENTURE OFFERING
CANADA SETS OIL INDUSTRY AID PACKAGE
CANADA ENERGY MINISTER TO MAKE STATEMENT

GM &lt;GM> CANADA AND WORKERS FAR APART IN TALKS
GM CANADA, WORKERS FAR APART IN TALKS - UNION
WESTCOAST &lt;WTC> TO ACQUIRE AGIP PROPERTIES
DOLLAR STEADY IN EUROPE AHEAD OF WASHINGTON TALKS
MULFORD SAYS G-6 WANTS STABILITY
BRITISH BANKS RESIST SIGNING MEXICO PACKAGE
LATIN OIL PRODUCERS TO MEET IN CARACAS
ENCOR PLANS 125 MLN DLR DEBENTURE OFFERING
CANADA SETS OIL INDUSTRY AID PACKAGE
CANADA ENERGY MINISTER TO MAKE STATEMENT